# 4. Ensembles de Arboles de Decision

## 4.03 Random Forest

*Random Forest* es un algoritmo de ensembles de arboles de decision creado por Leo Brieman en 1995-2006
https://link.springer.com/content/pdf/10.1023/a:1010933404324.pdf

La página original es:
https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm

Dos buenos videos para seguir el paso a paso de Random Forest y aplicaciones:
* https://www.youtube.com/watch?v=J4Wdy0Wc_xQ
* https://www.youtube.com/watch?v=sQ870aTKqiM

Qué tipo de perturbaciones se realizan en Random Forest

*   Se perturba el dataset, con la técnica de bagging = Bootstrap Aggregating
*   Tambien se perturba el algoritmo, utiliza random en cada split

Cada arbolito de Random Forest se entrena sobre un dataset perturbado, que tiene :
* todas las columnas originales (esta es una GRAN diferencia con  Arboles Azarosos)
* la misma *cantidad* de registros del dataset original, PERO generados por la técnica de sampleo con reposición del dataset original.

A pesar de que Leo Brieman es también el inventor de CART (Classification and Regression Trees) Random Forest no corre el algoritmo CART de la libreria rpart, sino un CART perturbado, en donde cada split NO se hace sobre todos los campos del dataset, sino sobre un csubconjunto tomado al azar, esa cantidad es el hiperparámetro *mtry*

#### 4.03.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   Bajar el  **competencia_01_crudo**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_01_crudo.csv"
archivo_destino="/content/datasets/competencia_01_crudo.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/competencia_01_crudo.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


## Generacion de la clase_ternaria

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

In [56]:
require( "data.table" )

# leo el dataset
dataset <- fread("../../Competencia 01/competencia_01_crudo.csv" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
    "pos" = .I,
    numero_de_cliente,
    periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 ) ]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]


# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

fwrite( dataset,
    file =  "./competencia_01.csv.gz",
    sep = ","
)

In [57]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
202101,BAJA+1,622
202101,BAJA+2,825
202101,CONTINUA,160080
202102,BAJA+1,831
202102,BAJA+2,1032
202102,CONTINUA,160292
202103,BAJA+1,1039
202103,BAJA+2,951
202103,CONTINUA,161119


### 4.04  Random Forest, una corrida

El tiempo de corrida de este punto es de alrededor de 8 minutos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [58]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "vie sept 05 22:06:48 2025"

In [79]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2462754,131.6,6935111,370.4,6935111,370.4
Vcells,4443344,34.0,109156455,832.8,388969928,2967.7


**ranger** es una de las muchas librerías en lenguage R que implementa el algoritmo *Random Forest*, tiene la ventaja que corre el paralelo, utilizando todos los nucleos del procesador.

In [ ]:
# cargo las librerias que necesito
if(!require("data.table")) install.packages("data.table")
require("data.table")

if(!require("rpart")) install.packages("rpart")
require("rpart")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

# ranger se usa para procesar
if( !require("ranger") ) install.packages("ranger")
require("ranger")

# randomForest  solo se usa para imputar nulos
if( !require("randomForest") ) install.packages("randomForest")
require("randomForest")

Aqui debe cargar SU semilla primigenia y

In [96]:
PARAM <- list()
PARAM$experimento <- 440
PARAM$semilla_primigenia <- 200003

# training y future
PARAM$train <- c(202102)
PARAM$future <- c(202104)

PARAM$ranger$num.trees <- 307 # cantidad de arboles
PARAM$ranger$mtry <- 14 # cantidad de atributos que participan en cada split
PARAM$ranger$min.node.size <- 336 # tamaño minimo de las hojas
PARAM$ranger$max.depth <- 17 # 0 significa profundidad infinita

PARAM$semilla_kaggle <- 314159

In [97]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

In [98]:
# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )

  return( drealidad )
}

In [99]:
# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}

In [101]:
# lectura del dataset
dataset <- fread("./competencia_01.csv.gz")

In [102]:
#  estas dos lineas estan relacionadas con el Data Drifting
# asigno un valor muy negativo

if( "Master_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Master_Finiciomora) , Master_Finiciomora := -999 ]

if( "Visa_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Visa_Finiciomora) , Visa_Finiciomora :=  -999 ]


In [103]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes %in% PARAM$train]

In [104]:
# mes donde voy a aplicar el modelo
dfuture <- dataset[foto_mes %in% PARAM$future]
setorder(dfuture, numero_de_cliente, foto_mes)

In [105]:
# inicilizo el dataset  drealidad
drealidad <- realidad_inicializar( dfuture, PARAM)

In [106]:
# quito clase ternaria de donde voy a aplicar el modelo
dfuture[, clase_ternaria:= NULL]

In [107]:
set.seed(PARAM$semilla_primigenia,"L'Ecuyer-CMRG" ) # Establezco la semilla aleatoria

# ranger necesita la clase de tipo factor
factorizado <- as.factor(dtrain$clase_ternaria)
dtrain[, clase_ternaria := factorizado]

In [108]:
# Ranger NO acepta valores nulos
# Leo Breiman, ¿por que le temias a los nulos?
# imputo los nulos, ya que ranger no acepta nulos
dtrain <- na.roughfix(dtrain)

In [109]:
setorder(dtrain, clase_ternaria) # primero quedan los BAJA+1, BAJA+2, CONTINUA

# genero el modelo de Random Forest llamando a ranger()
modelo <- ranger(
  formula= "clase_ternaria ~ .",
  data= dtrain,
  probability= TRUE, # para que devuelva las probabilidades
  num.trees= PARAM$ranger$num.trees,
  mtry= PARAM$ranger$mtry,
  min.node.size= PARAM$ranger$min.node.size,
  max.depth= PARAM$ranger$max.depth
)


Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 25 seconds.
Growing trees.. Progress: 36%. Estimated remaining time: 1 minute, 51 seconds.
Growing trees.. Progress: 54%. Estimated remaining time: 1 minute, 20 seconds.
Growing trees.. Progress: 72%. Estimated remaining time: 48 seconds.
Growing trees.. Progress: 90%. Estimated remaining time: 18 seconds.


In [110]:
# Carpinteria necesaria sobre  dfuture
# como quiere la Estadistica Clasica, imputar nulos por separado
# ( aunque en este caso ya tengo los datos del futuro de antemano
#  pero bueno, sigamos el librito de estos fundamentalistas a rajatabla ...

dfuture <- na.roughfix(dfuture)

In [111]:
tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]

In [112]:
# aplico el modelo a los datos que no tienen clase
# aplico el modelo recien creado a los datos del futuro
prediccion <- predict(modelo, dfuture)

tb_prediccion[, prob := prediccion$predictions[, "BAJA+2"] ]

In [113]:
tb_prediccion[, Predicted := as.numeric(prob > (1/40))]

In [114]:
res <- realidad_evaluar( drealidad, tb_prediccion)

In [115]:
cat( " TOTAL=", res$total,
  " Public=", res$public,
  " Private=", res$private,
  "\n",
  sep= ""
)

 TOTAL=299720000 Public=275933333 Private=309914286


In [116]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "vie sept 05 22:22:28 2025"

Reportar el resultado  TOTAL en la hoja C4-Random Forest de la planilla colaborativa

Usted NO reportará la ganancia del Public ni tampoco la del Private, por ahora es simplemente para que perciba la variabilidad existente y comience a tener plena conciencia de los fenómenos que observó en la asignatura  Data Mining del cuatrimestre anterior.



---



### 4.05  Random Forest  optimizacion de hiperparámetros

Random Forest es un algoritmo que quedó obsoleto luego de la aparición de  XGBoost y LightGBM, debido a lo lento de las librerías que lo implementan.
<br> El siguiente script se brinda simplemente a modo pedagógico, advirtiendo a l@s alumn@s que demanda más de 24 horas para correr, y los resultados no son brillantes.

limpio el ambiente de R

In [17]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "vie sept 05 10:43:17 2025"

In [18]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2132210,113.9,4164375,222.5,2968170,158.6
Vcells,3784812,28.9,8388608,64.0,8386173,64.0


**ranger** es una de las muchas librerías en lenguage R que implementa el algoritmo *Random Forest*, tiene la ventaja que corre el paralelo, utilizando todos los nucleos del procesador.

In [ ]:
# cargo las librerias que necesito
if(!require("data.table")) install.packages("data.table")
require("data.table")

if(!require("rpart")) install.packages("rpart")
require("rpart")

if(!require("parallel")) install.packages("parallel")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

# ranger se usa para procesar
if( !require("ranger") ) install.packages("ranger")
require("ranger")

# randomForest  solo se usa para imputar nulos
if( !require("randomForest") ) install.packages("randomForest")
require("randomForest")


if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Aqui debe cargar SU semilla primigenia y

In [20]:
PARAM <- list()
PARAM$experimento <- 450
PARAM$semilla_primigenia <- 200003

PARAM$hyperparametertuning$iteraciones <- 40
PARAM$hyperparametertuning$xval_folds <- 5
PARAM$hyperparametertuning$POS_ganancia <- 780000
PARAM$hyperparametertuning$NEG_ganancia <- -20000

# Estructura que define los hiperparámetros y sus rangos
#  la letra L al final significa ENTERO
# max.depth 0 significa profundidad infinita
PARAM$hyperparametertuning$hs <- makeParamSet(
  makeIntegerParam("num.trees", lower= 20L, upper= 500L),
  makeIntegerParam("max.depth", lower= 1L, upper= 30L),
  makeIntegerParam("min.node.size", lower= 1L, upper= 1000L),
  makeIntegerParam("mtry", lower= 2L, upper= 50L)
)

# training
PARAM$train <- c(202102)

In [21]:
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(
    reg, arch= NA, folder= "./",
    ext= ".txt", verbose= TRUE) {

  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  if (!file.exists(archivo)) # Escribo los titulos
    {
      linea <- paste0(
        "fecha\t",
        paste(list.names(reg), collapse= "\t"), "\n"
      )

      cat(linea, file= archivo)
    }

  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t", # la fecha y hora
    gsub(", ", "\t", toString(reg)), "\n"
  )

  cat(linea, file= archivo, append= TRUE) # grabo al archivo

  if (verbose) cat(linea) # imprimo por pantalla
}


In [22]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30
# particionar( data=dataset, division=c(1,1,1,1,1),
#   agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar <- function(
    data, division, agrupa= "",
    campo= "fold", start= 1, seed= NA) {

  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by= agrupa
  ]
}


In [23]:
# es un paso del Cross Validation
# utiliza el fold  fold_test para testear y el resto para entrenar

ranger_Simple <- function(fold_test, pdata, param) {
  # genero el modelo

  set.seed(PARAM$semillas[2])

  modelo <- ranger(
    formula= "clase_binaria ~ .",
    data= pdata[fold != fold_test],
    probability= TRUE, # para que devuelva las probabilidades
    num.trees= param$num.trees,
    mtry= param$mtry,
    min.node.size= param$min.node.size,
    max.depth= param$max.depth
  )

  prediccion <- predict(modelo, pdata[fold == fold_test])

  ganancia_testing <- pdata[
    fold == fold_test,
    sum((prediccion$predictions[, "POS"] > 1 / 40) *
      ifelse(clase_binaria == "POS",
        PARAM$hyperparametertuning$POS_ganancia,
        PARAM$hyperparametertuning$NEG_ganancia
      ))
  ]

  return(ganancia_testing)
}


In [24]:
# realiza Cross Validation, promediando las ganancias de los folds de testing

ranger_CrossValidation <- function(
    data, param,
    pcampos_buenos, qfolds, pagrupa, semilla) {

  divi <- rep(1, qfolds)
  particionar(data, divi, seed= semilla, agrupa= pagrupa)

  ganancias <- mcmapply(ranger_Simple,
    seq(qfolds), # 1 2 3 4 5
    MoreArgs= list(data, param),
    SIMPLIFY= FALSE,
    mc.cores= 1
  ) # dejar esto en  1, porque ranger ya corre en paralelo

  data[, fold := NULL] # elimino el campo fold

  # devuelvo la ganancia promedio normalizada
  ganancia_promedio <- mean(unlist(ganancias))
  ganancia_promedio_normalizada <- ganancia_promedio * qfolds

  return(ganancia_promedio_normalizada)
}

In [25]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros se pasan como variables globales

EstimarGanancia_ranger <- function(x) {
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  xval_folds <- PARAM$hyperparametertuning$xval_folds

  ganancia <- ranger_CrossValidation(dataset,
    param= x,
    qfolds= xval_folds,
    pagrupa= "clase_binaria",
    semilla= PARAM$semillas[1]
  )

  # logueo
  xx <- x
  xx$xval_folds <- xval_folds
  xx$ganancia <- ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch= klog)

  # si es ganancia superadora la almaceno en mejor
  if( ganancia > GLOBAL_mejor ) {
    GLOBAL_mejor <<- ganancia
    loguear(xx, arch= klog_mejor)
  }


  return(ganancia)
}


aqui se inicia el programa

In [26]:
# genero numeros primos
primos <- generate_primes(min= 100000, max= 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, 2 )

In [ ]:
# lectura del dataset
dataset <- fread("../competencia_01.csv.gz")

In [28]:
# solo trabajo con  training
dataset <- dataset[foto_mes %in% PARAM$train ]

In [29]:
#  estas dos lineas estan relacionadas con el Data Drifting
# asigno un valor muy negativo

if( "Master_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Master_Finiciomora) , Master_Finiciomora := -999 ]

if( "Visa_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Visa_Finiciomora) , Visa_Finiciomora :=  -999 ]


In [30]:
set.seed(PARAM$semilla_primigenia,"L'Ecuyer-CMRG" ) # Establezco la semilla aleatoria

In [31]:
# en estos archivos quedan los resultados
kbayesiana <- paste0("HT", PARAM$experimento, ".RDATA")
klog <- paste0("HT", PARAM$experimento, ".txt")
klog_mejor <- paste0("HT", PARAM$experimento, "_mejor.txt")

GLOBAL_iteracion <- 0 # inicializo la variable global
GLOBAL_mejor <- -Inf

# si ya existe el archivo log, traigo hasta donde llegue
if (file.exists(klog)) {
  tabla_log <- fread(klog)
  GLOBAL_iteracion <- nrow(tabla_log)
}


In [32]:
# paso a trabajar con clase binaria POS={BAJA+2}   NEG={BAJA+1, CONTINUA}
dataset[, clase_binaria :=
  as.factor(ifelse(clase_ternaria == "BAJA+2", "POS", "NEG"))]

dataset[, clase_ternaria := NULL] # elimino la clase_ternaria, ya no la necesito


In [33]:
# Ranger NO acepta valores nulos
# Leo Breiman, ¿por que le temias a los nulos?
# imputo los nulos, ya que ranger no acepta nulos

dataset <- na.roughfix(dataset)

In [34]:
# Aqui comienza la configuracion de la Bayesian Optimization

configureMlr(show.learner.output = FALSE)

funcion_optimizar <- EstimarGanancia_ranger

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar,
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hyperparametertuning$hs,
  has.simple.signature= FALSE
)

ctrl <- makeMBOControl(save.on.disk.at.time= 600, save.file.path= kbayesiana)

ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
)

ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


In [35]:
# inicio la optimizacion bayesiana

if (!file.exists(kbayesiana)) {
  run <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  run <- mboContinue(kbayesiana)
} # retomo en caso que ya exista

Computing y column(s) for design. Not provided.



Growing trees.. Progress: 17%. Estimated remaining time: 2 minutes, 34 seconds.
Growing trees.. Progress: 34%. Estimated remaining time: 2 minutes, 2 seconds.
Growing trees.. Progress: 52%. Estimated remaining time: 1 minute, 29 seconds.
Growing trees.. Progress: 69%. Estimated remaining time: 55 seconds.
Growing trees.. Progress: 87%. Estimated remaining time: 23 seconds.
Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 19 seconds.
Growing trees.. Progress: 37%. Estimated remaining time: 1 minute, 48 seconds.
Growing trees.. Progress: 55%. Estimated remaining time: 1 minute, 17 seconds.
Growing trees.. Progress: 73%. Estimated remaining time: 45 seconds.
Growing trees.. Progress: 91%. Estimated remaining time: 15 seconds.
Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 24 seconds.
Growing trees.. Progress: 36%. Estimated remaining time: 1 minute, 50 seconds.
Growing trees.. Progress: 54%. Estimated remaining time: 1 minute, 21 seconds.
Growing tr

[mbo] 0: num.trees=215; max.depth=15; min.node.size=421; mtry=36 : y = 2.32e+08 : 918.0 secs : initdesign

[mbo] 0: num.trees=247; max.depth=19; min.node.size=279; mtry=35 : y = 2.24e+08 : 1198.3 secs : initdesign

[mbo] 0: num.trees=355; max.depth=26; min.node.size=575; mtry=8 : y = 2.45e+08 : 419.0 secs : initdesign

[mbo] 0: num.trees=329; max.depth=12; min.node.size=481; mtry=29 : y = 2.41e+08 : 975.3 secs : initdesign

[mbo] 0: num.trees=427; max.depth=20; min.node.size=166; mtry=7 : y = 2.4e+08 : 485.1 secs : initdesign

[mbo] 0: num.trees=119; max.depth=22; min.node.size=503; mtry=24 : y = 2.28e+08 : 402.8 secs : initdesign

[mbo] 0: num.trees=466; max.depth=6; min.node.size=116; mtry=39 : y = 2.24e+08 : 789.7 secs : initdesign

[mbo] 0: num.trees=36; max.depth=24; min.node.size=742; mtry=5 : y = 2.3e+08 : 36.2 secs : initdesign

[mbo] 0: num.trees=57; max.depth=8; min.node.size=927; mtry=46 : y = 2.16e+08 : 160.5 secs : initdesign

[mbo] 0: num.trees=150; max.depth=1; min.node.

Growing trees.. Progress: 5%. Estimated remaining time: 9 minutes, 3 seconds.
Growing trees.. Progress: 11%. Estimated remaining time: 8 minutes, 11 seconds.
Growing trees.. Progress: 17%. Estimated remaining time: 7 minutes, 40 seconds.
Growing trees.. Progress: 22%. Estimated remaining time: 7 minutes, 18 seconds.
Growing trees.. Progress: 28%. Estimated remaining time: 6 minutes, 41 seconds.
Growing trees.. Progress: 33%. Estimated remaining time: 6 minutes, 14 seconds.
Growing trees.. Progress: 39%. Estimated remaining time: 5 minutes, 45 seconds.
Growing trees.. Progress: 44%. Estimated remaining time: 5 minutes, 18 seconds.
Growing trees.. Progress: 50%. Estimated remaining time: 4 minutes, 47 seconds.
Growing trees.. Progress: 55%. Estimated remaining time: 4 minutes, 18 seconds.
Growing trees.. Progress: 60%. Estimated remaining time: 3 minutes, 47 seconds.
Growing trees.. Progress: 66%. Estimated remaining time: 3 minutes, 16 seconds.
Growing trees.. Progress: 71%. Estimated r

[mbo] 1: num.trees=500; max.depth=16; min.node.size=1000; mtry=48 : y = 2.27e+08 : 2850.1 secs : infill_ei

Saved the current state after iteration 2 in the file HT450.RDATA.

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
"generateDesign could only produce 999 points instead of 1000!"
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
"generateDesign could only produce 924 points instead of 1000!"


...

[mbo] 38: num.trees=57; max.depth=14; min.node.size=1; mtry=50 : y = 1.9e+08 : 324.1 secs : infill_ei

Saved the current state after iteration 39 in the file HT450.RDATA.



Growing trees.. Progress: 21%. Estimated remaining time: 1 minute, 53 seconds.
Growing trees.. Progress: 43%. Estimated remaining time: 1 minute, 21 seconds.
Growing trees.. Progress: 65%. Estimated remaining time: 50 seconds.
Growing trees.. Progress: 86%. Estimated remaining time: 20 seconds.
Growing trees.. Progress: 21%. Estimated remaining time: 1 minute, 56 seconds.
Growing trees.. Progress: 42%. Estimated remaining time: 1 minute, 23 seconds.
Growing trees.. Progress: 64%. Estimated remaining time: 51 seconds.
Growing trees.. Progress: 86%. Estimated remaining time: 20 seconds.
Growing trees.. Progress: 21%. Estimated remaining time: 1 minute, 54 seconds.
Growing trees.. Progress: 43%. Estimated remaining time: 1 minute, 23 seconds.
Growing trees.. Progress: 64%. Estimated remaining time: 51 seconds.
Growing trees.. Progress: 86%. Estimated remaining time: 20 seconds.
Growing trees.. Progress: 22%. Estimated remaining time: 1 minute, 51 seconds.
Growing trees.. Progress: 43%. Es

[mbo] 39: num.trees=433; max.depth=29; min.node.size=452; mtry=13 : y = 2.52e+08 : 779.2 secs : infill_ei

Saved the current state after iteration 40 in the file HT450.RDATA.



Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 25 seconds.
Growing trees.. Progress: 35%. Estimated remaining time: 1 minute, 55 seconds.
Growing trees.. Progress: 53%. Estimated remaining time: 1 minute, 23 seconds.
Growing trees.. Progress: 71%. Estimated remaining time: 51 seconds.
Growing trees.. Progress: 89%. Estimated remaining time: 19 seconds.
Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 21 seconds.
Growing trees.. Progress: 36%. Estimated remaining time: 1 minute, 49 seconds.
Growing trees.. Progress: 54%. Estimated remaining time: 1 minute, 18 seconds.
Growing trees.. Progress: 73%. Estimated remaining time: 46 seconds.
Growing trees.. Progress: 91%. Estimated remaining time: 15 seconds.
Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 25 seconds.
Growing trees.. Progress: 36%. Estimated remaining time: 1 minute, 52 seconds.
Growing trees.. Progress: 54%. Estimated remaining time: 1 minute, 19 seconds.
Growing tr

[mbo] 40: num.trees=439; max.depth=18; min.node.size=457; mtry=16 : y = 2.44e+08 : 913.8 secs : infill_ei

Saved the final state in the file HT450.RDATA

Saved the final state in the file HT450.RDATA



In [36]:
# analizo la salida de la bayesiana

tb_bayesiana <- fread(klog)
setorder( tb_bayesiana, -ganancia)
tb_bayesiana

fecha,num.trees,max.depth,min.node.size,mtry,xval_folds,ganancia,iteracion
<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>
20250905 161459,307,17,336,14,5,252560000,25
20250905 212917,433,29,452,13,5,251580000,55
20250905 171018,378,18,410,17,5,248040000,32
20250905 165616,307,25,801,16,5,247560000,31
20250905 182556,401,17,343,9,5,247500000,39
20250905 191508,293,19,503,13,5,246880000,46
20250905 180130,500,30,617,17,5,246700000,37
20250905 185349,260,30,630,12,5,245360000,44
20250905 143859,500,14,312,18,5,245160000,19


In [37]:
# mejores parametros

print( tb_bayesiana[1] )

             fecha num.trees max.depth min.node.size  mtry xval_folds  ganancia
            <char>     <int>     <int>         <int> <int>      <int>     <num>
1: 20250905 161459       307        17           336    14          5 252560000
   iteracion
       <int>
1:        25


In [38]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "vie sept 05 21:45:06 2025"

Una vez que se impriman los mejores hiperparametros proceder:


1.   Copielos en el punto 4.04  en la celda donde se define PARAM
2.   Ejecute el punto 4.04 con estos hiperparámetros optimos
2.   Reporte el resultado en la hoja  **C4-RF Bayesiana**  de la  planilla colaborativa





---

